## Minimally complex models for supreme court voting dataset

In [156]:
# Imports

import sys
sys.path.insert(0, '../')
from src.utils import *
from src.mcm import mcm

In [157]:
model = mcm('../data/SC_voting/US_SupremeCourt_n9_N895.txt')

In [158]:
# Evidence of IM with original basis (Should be -5258)
print('Evidence: ', model.calc_log_evidence(model.mcms[-1]))

Evidence:  -5258.100240438084


In [6]:
# Finding best MCM in original basis (Should be [[0, 2, 3, 4, 6], [1, 5, 7, 8]] with evidence -3300.4)
model.find_best_mcm()
print('MCM: ', model.best_mcm)
print('Evidence: ', model.best_evidence)

MCM:  [[0, 2, 3, 4, 6], [1, 5, 7, 8]]
Evidence:  -3300.395469673639


In [159]:
# Finding the best IM (Should be [72, 160, 384, 17, 65, 5, 130, 260, 64] with evidence -3327)
model.find_best_im()
print('IM: ', model.best_im)
model.transform_data()
print('Evidence: ', model.calc_log_evidence(model.mcms[-1]))

IM:  [72, 160, 384, 17, 65, 5, 130, 260, 64]
Evidence:  -3327.079945846724


In [9]:
# Finding best MCM in optimal basis (Should be [[0], [1, 2, 6], [3, 4, 5, 7, 8]] with evidence -3154.4)
model.find_best_mcm()
print('MCM: ', model.best_mcm)
print('Evidence: ', model.best_evidence)

MCM:  [[0], [1, 2, 6], [3, 4, 5, 7, 8]]
Evidence:  -3154.421230299754


#### Greedy search

In [142]:
# Generate all operators upto order k
k = 2
all_ops = generate_ops_upto_order_n(model.n_var, k)

In [143]:
# Find the n most biased operator

# Array with the spinoperators in the best IM
im = []
# Complete model that can be formed with the IM 
model_im = []
for _ in range(model.n_var):
    max_bias = 0
    most_bias_op = all_ops[0]

    # Loop over all available operators
    for op in all_ops:
        # Calculate bias of the operator
        current_bias = model.calc_bias(op)
        # Check if this is max biased operator so far
        if current_bias > max_bias:
            most_bias_op = op
            max_bias = current_bias

    # Add most biased operator to the best IM
    im.append(most_bias_op)
    model_im.append(most_bias_op)
    all_ops = np.delete(all_ops, np.where(all_ops == most_bias_op))

    # Filter out operators that are not independent from operators in IM basis
    for op in model_im[:]:
        comb_op = op^most_bias_op
        model_im.append(comb_op)
        all_ops = np.delete(all_ops, np.where(all_ops == comb_op))

In [144]:
im

[72, 160, 384, 17, 65, 5, 130, 260, 64]

In [145]:
# GT of the data
model.transform_data(im)

In [146]:
all_ops = generate_ops_upto_order_n(model.n_var, k)
# Find the n most biased operator

# Array with the spinoperators in the best IM
im = []
# Complete model that can be formed with the IM 
model_im = []
for _ in range(model.n_var):
    max_bias = 0
    most_bias_op = all_ops[0]

    # Loop over all available operators
    for op in all_ops:
        # Calculate bias of the operator
        current_bias = model.calc_bias(op)
        # Check if this is max biased operator so far
        if current_bias > max_bias:
            most_bias_op = op
            max_bias = current_bias

    # Add most biased operator to the best IM
    im.append(most_bias_op)
    model_im.append(most_bias_op)
    all_ops = np.delete(all_ops, np.where(all_ops == most_bias_op))

    # Filter out operators that are not independent from operators in IM basis
    for op in model_im[:]:
        comb_op = op^most_bias_op
        model_im.append(comb_op)
        all_ops = np.delete(all_ops, np.where(all_ops == comb_op))

In [147]:
im

[1, 2, 4, 8, 16, 32, 64, 128, 256]

In [179]:
# Best MCM using hierarchical merging procedure

# Start with best IM
start_mcm = model.mcms[-1] 

best_ev = model.calc_log_evidence(start_mcm)


In [180]:
# Generate all combinations in which 2 ICCs are merged

best_mcm = [start_mcm[i] for i in range(len(start_mcm))]

while len(start_mcm) > 1:
    print(start_mcm, best_ev)
    n = len(start_mcm)
    for i in range(n):
        for j in range(i+1, n):
            # Start with current MCM
            new_mcm = [start_mcm[i] for i in range(len(start_mcm))]
            # Concatenate two ICCs
            new_mcm[i] = [k for l in [new_mcm[i], new_mcm[j]] for k in l]
            del new_mcm[j]

            ev = model.calc_log_evidence(new_mcm)
            if ev > best_ev:
                best_mcm = [new_mcm[i] for i in range(len(new_mcm))]
                best_ev = ev

    # Stop procedure if start MCM is the best one
    if best_mcm == start_mcm:
        break

    else:
        start_mcm = [best_mcm[i] for i in range(len(best_mcm))]


[[0], [1], [2], [3], [4], [5], [6], [7], [8]] -3327.079945846724
[[0], [1], [2], [3, 5], [4], [6], [7], [8]] -3288.1697657598957
[[0], [1, 2], [3, 5], [4], [6], [7], [8]] -3257.8933011552176
[[0], [1, 2, 6], [3, 5], [4], [7], [8]] -3228.3886812667183
[[0, 4], [1, 2, 6], [3, 5], [7], [8]] -3207.643778254081
[[0, 4], [1, 2, 6, 7], [3, 5], [8]] -3188.9849677119305
[[0, 4, 3, 5], [1, 2, 6, 7], [8]] -3172.0943946081766
[[0, 4, 3, 5], [1, 2, 6, 7, 8]] -3163.252527450342


In [186]:
model.calc_log_evidence([[0, 4, 3, 5, 8], [1, 2, 6, 7]])

-3176.3645149639074